### Exercise 10.1

In [ ]:
import numpy as np
import sys
from matplotlib import pyplot as plt
import time
import scipy.sparse as sparse
import scipy.sparse.linalg
ROOTDIR = '/home/lochy/Documents/Subject/ucd_geology/Gel298_winter/jupyter_notebooks'
sys.path.append('%s/python_scripts' % ROOTDIR)
from GMesh2d import MESH2D
import GHeatConservationSolver as GCS

#### Define the explicit solver

The explicit solver is formulated in one time step by eq 10.3 from the textbook:

$$ T_3 = T_3^0 + \frac{k \Delta t}{\rho C_p}\left(\frac{T_1^0 - 2 T_3^0 - T_5^0}{{\Delta x}^2} + \frac{T_2^0 - 2 T_3^0 - T_4^0}{{\Delta y}^2}\right)$$

Here are the position of the stencils

![](./stencil_energy_conservation.PNG.jpg)

#### Initiate a 2d mesh

This MESH2D class is defined in "python_scripts/GMesh2d.py". 
Here, we use a regular grid of 51 × 31 points. The model size is 1000 × 1500 km2 (i.e. 1 000 000 × 1 500 000 m2). 

In [ ]:
# initiate mesh
xsize = 1000000.0 # Model size, m
ysize = 1500000.0
xnum = 41   # Number of nodes
ynum = 61
xs = np.linspace(0.0, xsize, xnum) # construct xs
ys = np.linspace(0.0, ysize, ynum) # construct ys
Mesh2d = MESH2D(xs, ys)

#### Read in the initial temperature

The initial setup corresponds to a background temperature of 1000 K with a rectangular thermal wave (1300 K) in the middle (‘wave’ means sharp perturbation of the temperature field).

In [ ]:
# initial temperature
def rectangular_wave_temperature(x, y, xsize, ysize):
    '''
    temperature profile of a retangular wave in the middle
    '''
    dx = xsize / 3.0
    dy = ysize / 3.0
    if type(x) == float and type(y) == float:
      assert(x >= 0 and x <= xsize and y >= 0 and y <= ysize)
      if x > (xsize - dx) / 2.0 and x < (xsize + dx) / 2.0\
        and y > (ysize - dy) / 2.0 and y < (ysize + dy) / 2.0:
        T = 1300.0
      else:
        T = 1000.0
    elif type(x) == np.ndarray and type(y) == np.ndarray:
      assert(x.shape == y.shape)
      mask = (x > (xsize - dx) / 2.0) & (x < (xsize + dx) / 2.0)\
        & (y > (ysize - dy) / 2.0) & (y < (ysize + dy) / 2.0)
      T = np.ones(x.shape) * 1000.0
      T[mask] = 1300.0
    else:
      raise TypeError("Type of x or y is wrong (either float or numpy.ndarray")
    return T

xxs, yys = np.meshgrid(xs, ys)
Ts_init = rectangular_wave_temperature(xxs, yys, xsize, ysize)

# plot
fig, ax = plt.subplots()
h = ax.pcolor(xxs, yys, Ts_init)
ax.set_title('Initial temperature')
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
ax.invert_yaxis()
ax.axis('equal')
ax.set_xlim([0.0, xsize])
ax.set_ylim([0.0, ysize])
fig.colorbar(h, ax=ax, label='T (K)')

### Defined the solver

Here is the break down of the parameters in the previous eqaudtion.s

In [ ]:
class EXPLICIT_SOLVER():

    def __init__(self, mesh, **kwargs):
        '''
        Initiation.
        Inputs:
            mesh (MESH2D object)
            kwargs (dict):
                'use_constant_thermal_conductivity' - use constant thermal conductivity in the model
                    by default, this is False. This would affect the choices of solver used.
        '''
        self.use_constant_thermal_conductivity = kwargs.get('use_constant_thermal_conductivity', False)
        self.mesh = mesh
        Nx, Ny = mesh.get_number_of_points_xy()
        self.Nx = Nx
        self.Ny = Ny
        self.Ts = np.zeros((Nx * Ny))
        self.assembled = False
        self.solved = False
        self.t = 0.0
        pass

    def initial_temperature(self, Ts):
        '''
        set initial temperature
        Inputs:
            Ts (ndarray) - array of temperature
        '''
        assert(Ts.shape==(self.Ny, self.Nx))
        # We want Ts to first increment on y axis and then on x axis,
        # thus we want to first transpose Ts, so that y would be the
        # first to go when we rival the matrix
        self.Ts = Ts.T.reshape(self.Nx * self.Ny)
    
    def get_time(self):
        '''
        get time outputs
        return
            time (float)
        '''
        return self.t

    def assemble(self, thermal_conductivity, rho, cp, dt):
        '''
        assemble the equations
        Inputs:
            thermal_conductivity - thermal conductivities
            rho - densities
            cp - heat capacity
            dt (float): increment in time
        '''
        assert(self.assembled == False)
        self.dt = dt
        Ts = self.Ts.copy()
        xs, ys = self.mesh.get_coordinates()
        I = []  # These are indexed into the "left" matrix L
        J = []  # of the linear function Lx = R
        V = []  # equivalent to L(i, j) = v
        R = []
        if self.use_constant_thermal_conductivity:
            ### constant k, rho and cp
            kappa = thermal_conductivity / (rho * cp)
            i = 0  # incrememnt on i for I indexes
            for jx in range(self.Nx):
                for iy in range(self.Ny):
                    k1 = self.global_index(iy, jx - 1)   # index of the point to the left
                    k2 = self.global_index(iy - 1, jx)   # index of the point above
                    k3 = self.global_index(iy, jx)  # index of this point
                    k4 = self.global_index(iy + 1, jx)   # index of the point below
                    k5 = self.global_index(iy, jx + 1)   # index of the point to the right
                    if iy > 0 and iy < self.Ny-1 and jx > 0 and jx < self.Nx-1:
                        # internal points
                        dx = xs[jx + 1] - xs[jx]
                        dy = ys[iy + 1] - ys[iy]
                        Ts_new = Ts[k3] + kappa * dt * ( (Ts[k1] - 2 * Ts[k3] + Ts[k5]) / dx**2.0\
                             + (Ts[k2] - 2*Ts[k3] + Ts[k4]) / dy**2.0) # get the solution directly
                        I.append(i)
                        J.append(k3)
                        V.append(1)
                        R.append(Ts_new)
                    elif iy == 0:
                        # Hereby, the boundaries are handled here. The insulating boudnary conditions
                        # are used here. This is formulated by one point on the boundary and another interal point
                        # next to it.
                        # top boundary
                        I.append(i)
                        J.append(k3)
                        V.append(1)
                        I.append(i)
                        J.append(k4)
                        V.append(-1)
                        R.append(0.0)
                    elif iy == self.Ny - 1:
                        # bottom boundary
                        I.append(i)
                        J.append(k2)
                        V.append(1)
                        I.append(i)
                        J.append(k3)
                        V.append(-1)
                        R.append(0.0)
                    elif jx == 0:
                        # left boundary
                        I.append(i)
                        J.append(k3)
                        V.append(1)
                        I.append(i)
                        J.append(k5)
                        V.append(-1)
                        R.append(0.0)
                    elif jx == self.Nx - 1:
                        # right boundary
                        I.append(i)
                        J.append(k1)
                        V.append(1)
                        I.append(i)
                        J.append(k3)
                        V.append(-1)
                        R.append(0.0)
                    else:
                        raise IndexError("Index error found with iy = %d, jx = %d" % (iy, jx))
                    i += 1
            assert(i == self.Nx * self.Ny)
            # Finally, assemble the L matrix and the R vector.
            self.L = sparse.csr_matrix((V, (I, J)), shape=(self.Nx * self.Ny, self.Nx * self.Ny))
            self.R = np.array(R)
            self.assembled = True  # set the flags
            self.solved = False
    
    def solve(self):
        '''
        solve the linear equations
        '''
        assert(self.assembled==True)
        start = time.time()
        self.Ts = scipy.sparse.linalg.spsolve(self.L, self.R)
        self.solved = True  # reset the flags
        self.assembled = False
        self.t += self.dt
        end = time.time()
        time_elapse = end - start
        print("Temperature solver: %.4e s to solver" % time_elapse)
    
    def global_index(self, iy, jx):
        '''
        Get global index from the indexing along x and y, note the differences to Geyra's book
        as we start from index 0
        Inputs:
            iy (int) - y index
            jx (int) - x index
        '''
        return self.Ny * jx + iy

    def export(self):
        '''
        export results, in the form of meshed data
        Return:
            xxs (ndarray of float): x coordinates
            yys (ndarray of float): y coordinates
            Ts (ndarray of float): temperature
        '''
        xs, ys = self.mesh.get_coordinates()
        xxs, yys = np.meshgrid(xs, ys)
        # here, again, we want the increment to first follow the y axis.
        Ts = np.transpose(self.Ts.reshape((self.Nx, self.Ny)))
        return xxs, yys, Ts

#### Solve the problem

An explicit FD is used with Eq (10.4) from the textbook. 
Use constant thermal conductivity k = 3 W/(m K), density ρ = 3200 kg/m 3 and heat capacity CP = 1000 J/(kg K) for the entire model. 

The limit of time step is given by $t_m = \frac{\Delta x^2}{2\kappa}$, thus we first output this value below.

In [ ]:
year = 365 * 24 * 3600.0
thermal_conductivity = 3
rho = 3200.0
cp = 1000.0
kappa = thermal_conductivity / rho / cp  # thermal diffusivity
dt_m = (xsize / (xnum-1.0))**2.0 / (2*kappa)
print("Limit on time step: %.4e year" % (dt_m / year))

With this estimation, we first test a scenario where the time increment is choosen as $0.6 t_m$

In [ ]:
dt = dt_m / 3.0  # what Geyra used in his script.
# dt = 1.0  # debug: with this, the temperature profile should be the initial profile
total_step = 20
# initiate solver
HCSolver = EXPLICIT_SOLVER(Mesh2d, use_constant_thermal_conductivity=True)
# HCSolver = GCS.EXPLICIT_SOLVER(Mesh2d, use_constant_thermal_conductivity=True)
HCSolver.initial_temperature(Ts_init)  # use the rectangular perturbation as the initial temperature

for step in range(total_step):
    # assemble and solve
    HCSolver.assemble(thermal_conductivity, rho, cp, dt)
    HCSolver.solve()
    # export and plot
    xxs, yys, Ts = HCSolver.export()
    t = HCSolver.get_time()
    fig, ax = plt.subplots()
    h = ax.pcolor(xxs, yys, Ts)
    ax.set_title('Temperature, t = %.4e year' % (t/year))
    ax.set_xlabel('x (m)')
    ax.set_ylabel('y (m)')
    ax.set_xlim([0.0, xsize])
    ax.set_ylim([0.0, ysize])
    ax.axis('equal')
    ax.invert_yaxis()
    fig.colorbar(h, ax=ax, label='T (K)')